In [2]:
#https://github.com/UKPLab/sentence-transformers
## %pip install -U sentence-transformers

     |████████████████████████████████| 61 kB 902 kB/s eta 0:00:01
     |████████████████████████████████| 769 kB 4.4 MB/s eta 0:00:01
Processing /Users/piek/Library/Caches/pip/wheels/69/09/d1/bf058f7d6fa0ecba2ce7c66be3b8d012beb4bf61a6e0c101c0/sacremoses-0.0.43-py3-none-any.whl
  Using cached sentencepiece-0.1.91-cp37-cp37m-macosx_10_6_x86_64.whl (1.1 MB)
     |████████████████████████████████| 2.1 MB 5.9 MB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.5.1-py3-none-any.whl size=100385 sha256=2d758bab69fafeecccc3ed285288a51eac5475c159c18cf55b5147d924dcc12d
  Stored in directory: /Users/piek/Library/Caches/pip/wheels/8f/3e/e4/9552e3788603671ffc1c9ff56bfee684b36a88b21c05f0f91e
Successfully built sentence-transformers
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pickle
import numpy as np
import pandas as pd
import os, random, glob, json, nltk, re
from nltk.corpus import stopwords
import torch

from sentence_transformers import SentenceTransformer

from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neural_network import MLPClassifier

In [3]:
train_texts = ["Patient loopt wankel en bibbert.",
         "Patient is moe van traplopen",
        "Ze fiets elke dag naar de winkel"]
train_labels = ['l2', 'i2', 'i4']

test_texts = ["Patient is wankel en wiebelt.",
         "Ik ben uitgeput van een rondje op straat.",
        "De man gaat met de fiets naar zijn werk."]
test_labels = ['l2', 'i2', 'i4']

In [4]:
#https://www.sbert.net/docs/pretrained_models.html
distilbert_sentence_model = SentenceTransformer ('distiluse-base-multilingual-cased')

In [5]:
distilbert_training_vectors = []
for text in train_texts:
    sentence_embeddings = distilbert_sentence_model.encode(text)
    distilbert_training_vectors.append(sentence_embeddings)

distilbert_test_vectors = []
for text in test_texts:
    sentence_embeddings = distilbert_sentence_model.encode(text)
    distilbert_test_vectors.append(sentence_embeddings)

In [6]:
print(distilbert_training_vectors[0])

[ 2.07514502e-02 -2.28623841e-02 -5.05056977e-03 -3.82808670e-02
 -1.12001300e-01 -4.49262150e-02  5.75736500e-02 -6.87159002e-02
  1.93609968e-02 -1.86701789e-02 -2.62484848e-02  3.42791006e-02
 -1.45592010e-02  4.39459495e-02 -5.67155629e-02 -2.20868178e-02
 -3.27832475e-02  6.28070757e-02 -2.00314336e-02  5.59477098e-02
  4.41969074e-02 -5.29789180e-02 -4.19642888e-02  2.49513099e-03
 -9.92450789e-02 -1.20912388e-03 -4.43195961e-02 -1.42013207e-02
  3.63480896e-02 -3.17648239e-02 -4.06718673e-03  7.26833288e-03
 -2.83776317e-04  4.18068133e-02 -6.35279194e-02 -3.23335044e-02
 -2.00090348e-03  8.54749791e-03 -3.31182927e-02 -1.38494009e-02
 -7.02743754e-02  3.96748520e-02  5.88322384e-03  2.26890258e-02
 -2.94144060e-02 -3.28779966e-02 -6.40081391e-02 -2.10930444e-02
  3.34102325e-02 -2.12274045e-02 -4.05326337e-02  1.17076198e-02
 -1.29251154e-02  3.60183716e-02 -4.70145158e-02  4.98163626e-02
 -6.19483031e-02 -6.03217771e-03  2.15015020e-02 -5.56116179e-02
 -2.93095149e-02 -6.19335

In [7]:
BERT_classifier = LinearSVC(random_state=0, tol=1e-5)
BERT_classifier.fit(distilbert_training_vectors, train_labels)
SVM_predictions = list(BERT_classifier.predict(distilbert_test_vectors))
predicted_test_scores= BERT_classifier.decision_function(distilbert_test_vectors) 
print(classification_report(test_labels, SVM_predictions))

              precision    recall  f1-score   support

          i2       0.00      0.00      0.00         1
          i4       0.50      1.00      0.67         1
          l2       1.00      1.00      1.00         1

    accuracy                           0.67         3
   macro avg       0.50      0.67      0.56         3
weighted avg       0.50      0.67      0.56         3



/Users/piek/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Trying BERTJE in SentenceTransformer

Trying to load bertje into SentenceTransformer:
https://repo.telematika.org/project/ukplab_sentence-transformers/

I had to do the following:

1. place the download of 'Bertje" in the folder '~/.cache/torch/sentence_transformers' folder
2. add a '  "__version__": "0.1",' field to the config.json file
3. add an empty "modules.json" file:

```[

]```

With these changes I can load BERTJE in SentenceTransformer.

In [8]:
localBertje = '/Users/piek/.cache/torch/sentence_transformers/wietsedv.bert-base-ductch-cased'

In [9]:
sentence_transformer_model = SentenceTransformer(localBertje)

In [10]:
bertje_training_vectors = []
for text in train_texts:
    sentence_embeddings = sentence_transformer_model.encode(text)
    bertje_training_vectors.append(sentence_embeddings)

bertje_test_vectors = []
for text in test_texts:
    sentence_embeddings = sentence_transformer_model.encode(text)
    bertje_test_vectors.append(sentence_embeddings)

IndexError: list index out of range

In [ ]:
BERT_classifier = LinearSVC(random_state=0, tol=1e-5)
BERT_classifier.fit(bertje_training_vectors, train_labels)
SVM_predictions = list(BERT_classifier.predict(bertje_test_vectors))
predicted_test_scores= BERT_classifier.decision_function(bertje_test_vectors) 
print(classification_report(bertje_test_vectors, SVM_predictions))

## Similarity and Clustering

In [133]:
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import KMeans

embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Corpus with example sentences
corpus = ['A man is eating food.',
          'A man is eating a piece of bread.',
          'A man is eating pasta.',
          'The girl is carrying a baby.',
          'The baby is carried by the woman',
          'A man is riding a horse.',
          'A man is riding a white horse on an enclosed ground.',
          'A monkey is playing drums.',
          'Someone in a gorilla costume is playing a set of drums.',
          'A cheetah is running behind its prey.',
          'A cheetah chases prey on across a field.']

corpus_embeddings = embedder.encode(corpus)


100%|██████████| 245M/245M [00:21<00:00, 11.6MB/s] 


In [138]:
num_clusters = 5
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

print(cluster_assignment)

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")


[0 0 0 2 2 4 4 3 3 1 1]
Cluster  1
['A man is eating food.', 'A man is eating a piece of bread.', 'A man is eating pasta.']

Cluster  2
['A cheetah is running behind its prey.', 'A cheetah chases prey on across a field.']

Cluster  3
['The girl is carrying a baby.', 'The baby is carried by the woman']

Cluster  4
['A monkey is playing drums.', 'Someone in a gorilla costume is playing a set of drums.']

Cluster  5
['A man is riding a horse.', 'A man is riding a white horse on an enclosed ground.']



In [136]:
# Query sentences:
queries = ['A man is eating pasta.', 'Someone in a gorilla costume is playing a set of drums.', 'A cheetah chases prey on across a field.']


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()

    #We use np.argpartition, to only partially sort the top_k results
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx in top_results[0:top_k]:
        print(corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))





Query: A man is eating pasta.

Top 5 most similar sentences in corpus:
A man is eating pasta. (Score: 1.0000)
A man is eating food. (Score: 0.5777)
A man is eating a piece of bread. (Score: 0.4986)
A man is riding a horse. (Score: 0.1581)
A man is riding a white horse on an enclosed ground. (Score: 0.1474)




Query: Someone in a gorilla costume is playing a set of drums.

Top 5 most similar sentences in corpus:
Someone in a gorilla costume is playing a set of drums. (Score: 1.0000)
A monkey is playing drums. (Score: 0.6435)
A man is eating a piece of bread. (Score: 0.1719)
A man is eating food. (Score: 0.1240)
A man is riding a white horse on an enclosed ground. (Score: 0.0706)




Query: A cheetah chases prey on across a field.

Top 5 most similar sentences in corpus:
A cheetah chases prey on across a field. (Score: 1.0000)
A cheetah is running behind its prey. (Score: 0.7769)
A man is riding a white horse on an enclosed ground. (Score: 0.2485)
A man is riding a horse. (Score: 0.